In [ ]:
import pandas as pd
import numpy as np
import os

import pickle
import time

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

import lightgbm as lgb
from lightgbm import LGBMClassifier

seed = 66

In [ ]:
# Case of class imbalance - sampling and shuffling data
validation_split = 0.8
oversampling_factor = 3
np.random.seed(seed)

# Assuming binary classification with integer 0 and 1 for lables
train_count = int(X['label'].sum() * validation_split)
valid_count = int(X['label'].sum() * (1-validation_split))

idx_1 = X.loc[X['label'] == 1].index.tolist()
np.random.shuffle(idx_1)
validation_index_1 = idx_1[:valid_count]
train_index_1 = idx_1[valid_count:]

idx_0 = X.loc[X['label'] == 0].index.tolist()
np.random.shuffle(idx_0)
validation_index_0 = idx_0[:valid_count] 
train_index_0 = idx_0[valid_count:]

X_Valid = pd.concat([X.loc[validation_index_1], X.loc[validation_index_0]])

X_tr_1 = X.loc[train_index_1].sample(train_count * oversampling_factor, replace=True)
X_tr_0 = X.loc[train_index_0].sample(train_count * oversampling_factor, replace=True)
X_Train = pd.concat([X_tr_1, X_tr_0])

In [ ]:
cols = [x for x in X_Train.columns if x != 'label']
train_x, train_y = X_Train[cols].values, X_Train['label'].values
valid_x, valid_y = X_Valid[cols].values, X_Valid['label'].values

preds = np.zeros(X_Valid.shape[0])
lgb_model = LGBMClassifier(
            nthread=4,
            n_estimators=1000,
            learning_rate=0.01,
            num_leaves=25, 
            silent=0,
            verbose=0,
            reg_alpha=0,
            reg_labmda=0)

lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
              eval_metric='auc', verbose=False)

lgb_model.booster_.save_model('lgb_sample.model', 
                              num_iteration=lgb_model.booster_.best_iteration)

feature_df = pd.DataFrame()
feature_df["feature"] = cols
feature_df["importance"] = clf.feature_importances_

In [ ]:
print('AUC score %.4f' %roc_auc_score(valid_y, lgb_model.predict_proba(valid_x)[:,1]))

pred_class = lgb_model.predict(valid_x)
conf_mat = confusion_matrix(y_true=valid_y, y_pred=pred_class)
print(conf_mat/cont_mat.sum())